Refactor the transformer model to produce 
1. (1) time (2) note plurality (3) motion
2. (1) note plurality (2) motion

First we'll try including the time

Create a vector representation of an output space detailing note categories as well as motions

Rewrite preprocess_transformer_data to process data in this fashion, perhaps automatically insert the contour arrays (or transformer representation of contour arrays)

note: let's replace "note category" with "note plurality"

In [2]:
from nltk.corpus import wordnet as wn

for ss in wn.synsets('multitude'):
    print(ss.name(), ss.lemma_names())



battalion.n.02 ['battalion', 'large_number', 'multitude', 'plurality', 'pack']
multitude.n.02 ['multitude', 'throng', 'concourse']
multitude.n.03 ['multitude', 'masses', 'mass', 'hoi_polloi', 'people', 'the_great_unwashed']


### Dataloader prototype

The dataloader will inheret from LazierDataset and ColabMemoryDataset   

In [4]:
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent.parent))
from tensor_hero.model import ColabMemoryDataset, LazierDataset, \
                              note_dirs_from_spec_dirs, check_notes_length
from tensor_hero.inference import __single_prediction_to_notes_array
from tensor_hero.preprocessing.data import encode_contour, notes_array_time_adjust
import os
from tqdm import tqdm
import numpy as np

class ContourMemoryDataset(ColabMemoryDataset):
    def __init__(self, partition_path, max_src_len, max_trg_len, max_examples, 
                 pad_idx, CHECK_LENGTH=False, tbps=25):
        self.max_trg_len = max_trg_len
        self.max_src_len = max_src_len
        self.pad_idx = pad_idx
        self.tbps = 25
        
        # Construct list of spectrogram file paths and list of note file paths
        song_paths = [partition_path / x for x in os.listdir(partition_path)]
        specs_dirs = [x / 'spectrograms' for x in song_paths]
        specs_lists = []
        for dir_ in specs_dirs:
            for specs_dir, _, specs in os.walk(dir_):
                if not specs:
                    continue
                specs_lists.append([Path(specs_dir) / spec for spec in specs])
        specs_lists = [spec for spec_list in specs_lists for spec in spec_list]  # Flatten
        notes_lists = [note_dirs_from_spec_dirs(x) for x in specs_lists]
        
        # Construct dictionary where key:value is <path to spec>:<path to notes array>
        l = {}  # labels
        for i in range(len(specs_lists)):
            l[specs_lists[i]] = notes_lists[i]
            
        # Weed out bits of data that exceed the maximum length
        self.labels = {}        # holds spec paths as keys, note paths as values
        self.data_paths = []    # list of spec paths
        too_long = 0            # how many of the notes have more elements than max_trg_len
        if CHECK_LENGTH:
            print('Checking length of spectrograms and notes...')
            for x in tqdm(specs_lists):
                if check_notes_length(l[x], max_trg_len):
                    self.data_paths.append(x)
                    self.labels[x] = l[x]
                else:
                    too_long += 1
                print(f'{too_long} datapoints removed due to exceeding maximum length')
        else:
            self.data_paths = specs_lists
            self.labels = l
            print('Notes were not checked against max_trg_len')
        
        # Restrict max samples in Dataset to min(max_examples, num_samples)        
        self.num_samples = len(self.labels)  # This could be lower than max_samples
        self.max_examples = max_examples if max_examples > 0 else self.num_samples
        self.max_examples = min(self.max_examples, self.num_samples)
        del too_long, l, song_paths, specs_dirs, specs_lists, notes_lists
        
        # Create and empty data matrix
        spec = np.load(self.data_paths[0])  # Load single examples to get shape
        notes = np.load(self.labels[self.data_paths[0]])
        # Shape for self.specs = [max_examples, 512, max_src_len]
        # Shape for self.notes = [max_examples, max_trg_len]
        self.specs = np.empty(shape=(self.max_examples, spec.shape[0], max_src_len))
        self.notes = np.empty(shape=(self.max_examples, max_trg_len))
        
        # Populate data into memory
        for idx in tqdm(range(self.max_examples)):
            spec = self.pad_spec(np.load(self.data_paths[idx]))
            # Transform notes into contour_vectors
            # contour_vectors are formatted to be transformer output
            notes = np.load(self.labels[self.data_paths[idx]])
            notes = self.contour_vector_from_notes(notes, tbps)
            notes = self.pad_notes(notes)
            self.specs[idx,...] = spec      # Final data
            self.notes[idx,...] = notes     # Final data
        print(f'self.specs (shape = {self.specs.shape}) is taking up {sys.getsizeof(self.specs) / (1024**2):.2f} MB')
        print(f'self.notes (shape = {self.notes.shape}) is taking up {sys.getsizeof(self.notes) / (1024**2):.2f} MB')
        del spec, notes
       
    def contour_vector_from_notes(self, notes, tbps):
        '''Captures original transformer output notes arrays and translates them to
        contour vectors

        Args:
            notes (1D numpy array): original transformer output formatted notes
            tbps (int): time bins per second represented in output array
        Returns:
            contour_vector (1D numpy array): transformer formatted contour array
                - [time, note plurality, motion, time, note plurality, motion, ...]
        '''
        notes_array = __single_prediction_to_notes_array(notes)

        # Reduce time bins per second from 100 to tbps
        notes_array, _ = notes_array_time_adjust(notes_array, time_bins_per_second=tbps)
        
        # Create contour
        contour = encode_contour(notes_array)
        
        # Convert to vector representation
        #      index         information
        #  0            | <sos> 
        #  1            | <eos> 
        #  2            | <pad> 
        #  3-15         | <note pluralities 0-13>
        #  16-24        | <motion [-4, 4]>
        #  25-(tbps+25) | <time bin 1-tbps>

         
         

SyntaxError: unexpected EOF while parsing (<ipython-input-4-f62202a540dd>, line 12)